### 2

In [1]:
import sys, requests, re, hashlib

hashlen = 5

url, auth = 'http://131.155.21.174:8081', ('', 'ilovecrypto')
sid, token = 20221886, '2635699b040f28285b99c543000fa18b4c895b8f'

sha = lambda m: int(hashlib.sha256((str(sid) + m).encode()).hexdigest()[:2*hashlen],16)

req = requests.Session()
req.cookies['token'] = token

r = req.get(url+'/', auth=auth)
n = int(re.search('n = ([0-9]+)', r.text).groups()[0])
e = int(re.search('e = ([0-9]+)', r.text).groups()[0])

def sign(m):
    r = req.post(url+'/sign', auth=auth, data={'msg': m})
    return int(r.text)

def validate(m, s):
    return pow(s, e, n) == sha(m)

def forgery(m, s):  # use this to submit your forgery once you've created it
    assert validate(m, s)
    r = req.post(url+'/validate_forgery', auth=auth, data={'msg': m, 'sig': str(s)})
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

In [2]:
print('public key: {}'.format((n,e)))

# let's sign a message and verify the signature

msg = "00000000000000000000000000000000000000Hi! Just trying this for fun"
print('message: {}'.format(repr(msg)))

val = sha(msg)
print('hash value: {}'.format(val))

sig = sign(msg)
print('signature: {}'.format(sig))
print('is valid? {}'.format(validate(msg, sig)))

public key: (152579285830235283227503885618892788961746910948104335839580651604160494241155242702334719934333460339845046895967236926672703967744310415286811741793540032355222144961745639991261008849867029346310388557481825441994609705068613777142239993109917812839610685937299142063270835695533679969730527395292945973567, 65537)
message: '00000000000000000000000000000000000000Hi! Just trying this for fun'
hash value: 170358956145
signature: 142829274821760819948729885194143352063928029083010484592453889814588997197919300382241806143853064848055714306431674450929832724855020075027655219160056840919895872936878383918505139682021526977321017393881657446625508182422174551130342107241865705781005069544560980747562902433830063004482009487916799291224
is valid? True


In [126]:
def getPrimes(bound):
    p = Primes()
    list_primes = []
    prime = p.first()
    while prime <= bound:
        list_primes.append(prime)
        prime = p.next(prime)
    list_primes = [int(prime) for prime in list_primes]
    return list_primes

S = getPrimes(1000)
l = len(S)

In [127]:
def bruteForce(n_msg, bound):
    max_length = 2^256
    suitable_msg = []
    while len(suitable_msg) < n_msg:
        m = randint(1, max_length)
        h = sha(str(m))        
        if max(list(factor(h)))[0] <= bound:
            if h not in suitable_msg:
                suitable_msg.append((m, h))
    return suitable_msg

In [218]:
H = bruteForce(l * 4, 1000)
J = H[0]

In [219]:
while J in H:
    J = bruteForce(1, 1000)[0] # returns an array, so [0]
J # The message for which we will forge the signature

(9342402959144252352375707487357706467916368572257867165551669044388447696802,
 227699366877)

In [220]:
H_factor = [factor(h[1]) for h in H]

In [221]:
H_factor[0]

2^4 * 103 * 173 * 379 * 809

In [222]:
p_place = {}
i = 0
# for ptime in list of Primes up to X (1000)
for p in S:
    p_place[p] = i
    i += 1
print(max(p_place))
print(p_place[997])

997
167


In [223]:
A = matrix(l * 4, l)

i = 0
for h in H_factor:
    for p in h:
        pos = p_place[p[0]]
        A[i, pos] = p[1]
    i += 1
A[0]
rank(A)
# len(A[0])

168

In [224]:
J_factor = factor(J[1])
J_factor

3^2 * 31 * 59 * 101 * 151 * 907

In [225]:
J_arr = vector([0 for i in range(l)])

In [226]:
for p in J_factor:
        pos = p_place[p[0]]
        J_arr[pos] = p[1] 
J_vector = vector(J_arr)
J_vector # V tao

(0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [227]:
f_e = GF(e)
AT = A.transpose()
AT = matrix(f_e,AT)
sol = AT.solve_right(vector(f_e, J_vector))
AT * sol == J_vector

True

In [228]:
B_sum

(6488165, 3473462, 1703963, 1179666, 1048592, 983055, 524296, 524296, 655370, 393222, 262148, 262148, 262148, 393222, 131074, 262148, 131074, 131074, 131074, 196611, 262149, 131074, 131074, 65537, 131074, 196611, 196611, 65537, 393222, 196612, 196611, 65537, 196611, 65537, 131074, 131074, 65537, 196611, 131074, 262148, 196611, 65537, 131074, 0, 0, 0, 65537, 131074, 131074, 131074, 131074, 65537, 65537, 131074, 65537, 196611, 65537, 131074, 131074, 65537, 65537, 131074, 65537, 65537, 131074, 65537, 65537, 65537, 131074, 196611, 0, 0, 0, 131074, 65537, 65537, 0, 0, 131074, 0, 131074, 0, 0, 65537, 0, 262148, 0, 0, 0, 65537, 0, 65537, 65537, 1, 0, 65537, 0, 65537, 65537, 196611, 65538, 0, 0, 65537, 0, 65537, 65537, 131074, 0, 65537, 0, 0, 65537, 65537, 0, 0, 0, 0, 131074, 0, 65537, 0, 65537, 65537, 0, 65537, 0, 0, 0, 0, 0, 65537, 0, 0, 0, 0, 65537, 0, 0, 65537, 0, 65537, 0, 65537, 65537, 0, 0, 65537, 65537, 0, 0, 65537, 65537, 0, 65537, 0, 65537, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65537, 65537)

In [229]:
B_sum = vector(ZZ, [0]*len(A[0]))
j = 0
for i in sol:
    B_sum += int(i) * vector(ZZ,A[j])
    j += 1
B_sum

(6422626, 4522055, 2687017, 1310740, 917518, 851981, 720907, 393222, 786444, 720907, 327686, 458759, 327685, 589833, 393222, 262148, 131075, 327685, 458759, 327685, 196611, 262148, 196611, 65537, 131074, 327686, 196611, 327685, 196611, 262148, 131074, 131074, 65537, 262148, 131074, 327686, 131074, 65537, 131074, 65537, 65537, 131074, 262148, 65537, 131074, 0, 131074, 65537, 65537, 65537, 0, 196611, 131074, 65537, 131074, 131074, 65537, 196611, 65537, 65537, 65537, 131074, 65537, 131074, 0, 131074, 65537, 131074, 0, 65537, 131074, 0, 65537, 131074, 65537, 0, 65537, 0, 0, 65537, 65537, 65537, 0, 0, 65537, 65537, 0, 65537, 0, 65537, 65537, 196611, 0, 65537, 0, 65537, 0, 65537, 0, 0, 131074, 0, 65537, 65537, 65537, 65537, 65537, 0, 0, 65537, 0, 65537, 65537, 65537, 65537, 0, 0, 65537, 0, 65537, 0, 65537, 131074, 0, 0, 0, 65537, 0, 0, 0, 0, 0, 0, 0, 0, 65537, 0, 0, 65537, 65537, 0, 131074, 0, 131074, 0, 0, 0, 65537, 0, 0, 0, 65537, 0, 0, 1, 65537, 65537, 0, 0, 0, 0, 65537, 65537, 131074, 0,

In [230]:
# print(type(A))
# print(type(B_sum))
# print(type(sol))
C = (J_vector - B_sum)/e
C

(-98, -69, -41, -20, -14, -13, -11, -6, -12, -11, -5, -7, -5, -9, -6, -4, -2, -5, -7, -5, -3, -4, -3, -1, -2, -5, -3, -5, -3, -4, -2, -2, -1, -4, -2, -5, -2, -1, -2, -1, -1, -2, -4, -1, -2, 0, -2, -1, -1, -1, 0, -3, -2, -1, -2, -2, -1, -3, -1, -1, -1, -2, -1, -2, 0, -2, -1, -2, 0, -1, -2, 0, -1, -2, -1, 0, -1, 0, 0, -1, -1, -1, 0, 0, -1, -1, 0, -1, 0, -1, -1, -3, 0, -1, 0, -1, 0, -1, 0, 0, -2, 0, -1, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, 0, -1, -2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, -2, 0, -2, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, -1, -1, -2, 0, -2, 0, -1)

In [231]:
C  = vector(ZZ,C)

In [232]:
# get signatures
G = []
for m in H:
    G.append(sign(m[0]))

In [233]:
for h, g in zip(H, G):
    # print(f"h {h}, g {g}")
    # print(f"type h {type(h[1])}")
    # print(f"type g {type(g)}")
    assert pow(g, e, n) == sha(str(h[0]))

In [238]:
left = 1
right = 1
i = 0
for prime in p_place:
    # left *= int(mod(prime ^ k[i], n))
    # Suppose Yj is not significant
    # left = int(mod(left * prime, n))
    # What the F is X supposed to be?
    Y = C[i]
    a = mod(Integer(prime) ** (Y), n)
    left = int(mod(left * a, n))
    i += 1
    
i = 0
for h in H:
    c = int(mod(G[i] ^ int(sol[i]), n))
    right = int(mod(right * c, n))
    i += 1
    

In [239]:
forge = int(mod(left * right, n))
forge

146878625056278049379910555675244221023902747211483804359058382191663912583496461648871235151245544917577693081026351796381860302361745833622952625841143717875694034626633386141196364985529099095588013992747213342562670184820789302196664023275711413552933229849214496020484325118261633675393119075479085889065

In [241]:
forgery(str(J[0]), forge)

Well done! We are accepting this signature as a forgery. The tale of your success has been saved to disk.


In [152]:
J[0]

58072293406276126704411965555725866915152013548805402697909249312505449581407